### part 2: Housing Prices

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv(r'C:\Users\laava\Desktop\sem 6\AOML\housing.csv')
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [3]:
df.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [4]:
df = df.dropna()
df.isnull().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

In [5]:
from sklearn.model_selection import train_test_split
import numpy as np

# Calculate the number of bins using Sturge's Rule
n = len(df)
k = int(1 + 3.322 * np.log10(n))

# Create bins for stratification based on 'median_income'
df['income_bin'] = pd.cut(df['median_income'], bins=k, labels=False)

# Perform stratified split
train_set, test_set = train_test_split(df, test_size=0.2, stratify=df['income_bin'], random_state=42)

# Drop the 'income_bin' column as it is no longer needed
train_set = train_set.drop('income_bin', axis=1)
test_set = test_set.drop('income_bin', axis=1)

train_set, test_set

C:\Users\laava\AppData\Local\Temp\ipykernel_25720\3226679149.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['income_bin'] = pd.cut(df['median_income'], bins=k, labels=False)


(       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
 12284    -116.81     33.90                17.0       2009.0           469.0   
 4811     -118.37     34.03                41.0       1425.0           285.0   
 6201     -117.89     34.08                25.0       2115.0           489.0   
 20126    -118.88     34.42                20.0        728.0           120.0   
 6188     -117.92     34.10                35.0       2994.0           603.0   
 ...          ...       ...                 ...          ...             ...   
 9101     -117.93     34.57                 5.0       5613.0          1060.0   
 2906     -119.03     35.38                52.0       1695.0           290.0   
 12405    -116.24     33.76                 9.0       1961.0           595.0   
 18137    -122.00     37.34                27.0       1716.0           290.0   
 8565     -118.42     33.92                41.0       1621.0           279.0   
 
        population  households  median

In [7]:
# Create dummy variables for the 'ocean_proximity' column in the training set
train_set = pd.get_dummies(train_set, columns=['ocean_proximity'], drop_first=True)

# Create dummy variables for the 'ocean_proximity' column in the test set
test_set = pd.get_dummies(test_set, columns=['ocean_proximity'], drop_first=True)

train_set.head(), test_set.head()

(       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
 12284    -116.81     33.90                17.0       2009.0           469.0   
 4811     -118.37     34.03                41.0       1425.0           285.0   
 6201     -117.89     34.08                25.0       2115.0           489.0   
 20126    -118.88     34.42                20.0        728.0           120.0   
 6188     -117.92     34.10                35.0       2994.0           603.0   
 
        population  households  median_income  median_house_value  \
 12284       820.0       381.0         1.3286             81800.0   
 4811        838.0       296.0         3.9732            188400.0   
 6201       1107.0       477.0         3.1949            207400.0   
 20126       360.0       115.0         6.1244            375000.0   
 6188       1933.0       561.0         4.0052            160700.0   
 
        ocean_proximity_INLAND  ocean_proximity_ISLAND  \
 12284                    True              

In [8]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Separate features and target variable from training set
X_train = train_set.drop('median_house_value', axis=1)
y_train = train_set['median_house_value']

# Separate features and target variable from test set
X_test = test_set.drop('median_house_value', axis=1)
y_test = test_set['median_house_value']

# Create a pipeline that standardizes the data then applies Ridge regression
ridge_pipeline = make_pipeline(StandardScaler(), Ridge(alpha=1.0))
ridge_pipeline.fit(X_train, y_train)

# Create a pipeline that standardizes the data then applies Lasso regression
lasso_pipeline = make_pipeline(StandardScaler(), Lasso(alpha=0.1))
lasso_pipeline.fit(X_train, y_train)

# Evaluate the models
ridge_score = ridge_pipeline.score(X_test, y_test)
lasso_score = lasso_pipeline.score(X_test, y_test)

ridge_score, lasso_score

(0.6499499258312675, 0.6499483713845833)

In [9]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Predict the target variable for the test set using both models
ridge_predictions = ridge_pipeline.predict(X_test)
lasso_predictions = lasso_pipeline.predict(X_test)

# Calculate MAE, MSE, and RMSE for Ridge regression
ridge_mae = mean_absolute_error(y_test, ridge_predictions)
ridge_mse = mean_squared_error(y_test, ridge_predictions)
ridge_rmse = np.sqrt(ridge_mse)

# Calculate MAE, MSE, and RMSE for Lasso regression
lasso_mae = mean_absolute_error(y_test, lasso_predictions)
lasso_mse = mean_squared_error(y_test, lasso_predictions)
lasso_rmse = np.sqrt(lasso_mse)

ridge_mae, ridge_mse, ridge_rmse, lasso_mae, lasso_mse, lasso_rmse

(49617.29375837566,
 4650497959.803248,
 68194.55960561112,
 49618.31042635085,
 4650518610.994799,
 68194.71101921907)